In [5]:
import os
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [10]:
!pip install --upgrade pandas

In [6]:
class Scraping:
    def _init_(self):
        # Define the directory where ChromeDriver will be stored
        chrome_driver_path = r'C:\Users\CHARITHA\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
        # Initialize the ChromeDriver
        chrome_options = webdriver.ChromeOptions()
        service = Service(chrome_driver_path)
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        self.START_DATE = "01/04/2018"
        self.END_DATE = "31/03/2023"
        self.BASE_URL = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?expandable=7"
        self.companies = []
        self.download_dir = r"c:\Users\CHARITHA\Downloads"
        self.company_files = {}
        self.company_sector={}
        self.file_count = 0
    def get_codes(self, file_name) -> list:
        companies = pd.read_csv(file_name)
        companies = companies.iloc[:,0].values.tolist()
        self.company = companies
        return companies[1000:2001]
           
    def singular_file_extraction(self, company: str) -> None:
        input_search = self.driver.find_element(By.ID, 'ContentPlaceHolder1_smartSearch')
        input_search.clear()
        input_search.send_keys(company)
        time.sleep(3)
        # Select the first child from the list of suggestions by class name
        first_suggestion = self.driver.find_element(By.CLASS_NAME, "quotemenu")  # Replace with the actual class name of the <li> element
        first_suggestion.click()
        s=self.driver.find_element(By.ID, "ContentPlaceHolder1_btnSubmit")
        self.driver.execute_script("arguments[0].click();",s)
        time.sleep(3)
        d=self.driver.find_element(By.ID, "ContentPlaceHolder1_btnDownload")
        self.driver.execute_script("arguments[0].click();", d)
        time.sleep(3)
        
        try:
            files = os.listdir(self.download_dir)
            paths = [os.path.join(self.download_dir, basename) for basename in files]
            most_recent_file = max(paths, key=os.path.getctime)
            most_recent_file_name = os.path.basename(most_recent_file)

            self.company_files[company] = most_recent_file_name
            print(f"Most recently downloaded file for {company}: {most_recent_file_name}")
        except Exception as e:
            print(f"Exception: {e}")
            print(f"Could not find most recently downloaded file for {company}")
            
    def get_sector(self):
        self.driver.get(self.BASE_URL)
        self.driver.maximize_window()
        editable_div = self.driver.find_elements(By.XPATH, '//input[@class="form-control hasDatepicker"]')
        self.driver.execute_script(f'arguments[0].setAttribute("value", "{self.START_DATE}")', editable_div[0])
        self.driver.execute_script(f'arguments[0].setAttribute("value", "{self.END_DATE}")', editable_div[1])
        time.sleep(5)


In [7]:
scraping = Scraping()
companies = scraping.get_codes('Equity (1).csv')
print(companies)
scraping.get_sector()
for company in companies:
    scraping.singular_file_extraction(company)  
    time.sleep(3)

scraping.driver.quit()

[514183, 514197, 514211, 514215, 514223, 514234, 514236, 514238, 514240, 514248, 514260, 514264, 514266, 514272, 514274, 514280, 514286, 514300, 514302, 514312, 514316, 514318, 514322, 514326, 514330, 514332, 514336, 514348, 514354, 514358, 514360, 514378, 514386, 514400, 514418, 514428, 514442, 514448, 514450, 514454, 514470, 515008, 515018, 515030, 515037, 515043, 515055, 515059, 515085, 515093, 515127, 515147, 516003, 516016, 516020, 516022, 516030, 516062, 516064, 516072, 516078, 516082, 516092, 516096, 516106, 516108, 516110, 517015, 517035, 517041, 517044, 517059, 517063, 517077, 517096, 517119, 517146, 517166, 517168, 517170, 517172, 517174, 517201, 517206, 517214, 517236, 517238, 517246, 517258, 517264, 517271, 517273, 517288, 517300, 517334, 517344, 517354, 517356, 517360, 517370, 517372, 517380, 517385, 517393, 517397, 517399, 517411, 517415, 517417, 517421, 517423, 517429, 517437, 517447, 517449, 517467, 517477, 517494, 517498, 517500, 517506, 517514, 517522, 517530, 517536,

AttributeError: 'Scraping' object has no attribute 'driver'

In [14]:
final_dataframe = pd.DataFrame()  
for company, file_name in scraping.company_files.items():
    if not file_name.endswith('.csv'):
        continue
    file_path = os.path.join(scraping.download_dir, file_name)
    df = pd.read_csv(file_path)
    df['Co Name'] = company
    final_dataframe = final_dataframe.append(df, ignore_index=True)
final_dataframe.to_csv(os.path.join(scraping.download_dir, 'final_dataset1_to_2.csv'), index=False)